In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Groupby apply on large (relational) data set

## Attentions all writen functions assume a data frame where the date is sorted!!

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


# Test data

In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]


In [4]:
test_data.head()


,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15791,2020-03-21,no,US,24508.0
16003,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33152.0
16268,2020-03-23,no,Germany,29056.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-09-09,no,256433.0
US,2020-09-09,no,6361265.0


In [6]:

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
   

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-09-09   256433.0
      US      2020-09-09  6361265.0

In [8]:
def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [9]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737            NaN
                16003            NaN
                16268       7.417994
                16534       7.142035
                16801       8.012983
                17067       6.954407
                17332       6.501919
                17599       7.390371
                17864      10.136671
                18131      13.541893
                18396      13.781393
                18663      13.140681
                18929      12.037271
                19195      12.735506
                19461      16.052694
                19726      21.372453
                19992      27.427355
                20258      27.511936
                20524      21.792179
                20791      21.495848
                21056      26.565108
                21322      36.198404
                21589      43.982990
                21855      49.423444
                22121      74.043747
                22387      56.424411
                

In [10]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [12]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [13]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


# Filtering the data with groupby apply 

In [14]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) 
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, 
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [15]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [16]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# Filtered doubling rate

In [17]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
61707,no,Zimbabwe,60476,72.397299
61708,no,Zimbabwe,60712,50.370316
61709,no,Zimbabwe,61008,54.005632
61710,no,Zimbabwe,61244,49.168665
61711,no,Zimbabwe,61711,42.255331


In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
61707,61707,2020-09-09,no,Barbados,180.0,359.333333,180.2,299.333333
61708,61708,2020-09-09,no,Belarus,73402.0,394.682839,73398.0,400.740558
61709,61709,2020-09-09,no,Belgium,89691.0,193.493131,89659.8,197.978695
61710,61710,2020-09-09,no,Albania,10704.0,70.834452,10704.4,70.267643
61711,61711,2020-09-09,no,Zimbabwe,7429.0,112.544529,7504.8,42.255331


In [19]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [20]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
60423,60423,2020-09-05,no,Germany,251058.0,225.482717,251107.0,209.504918
60691,60691,2020-09-06,no,Germany,251728.0,347.436678,252330.4,203.616986
60955,60955,2020-09-07,no,Germany,253626.0,196.368640,253560.4,205.700334
61223,61223,2020-09-08,no,Germany,254957.0,156.975534,254958.3,193.018278
61488,61488,2020-09-09,no,Germany,256433.0,181.692436,256356.2,182.386651


In [22]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)